In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.spatial import distance_matrix
import itertools
import scipy.sparse as sp
from tqdm import tqdm
import operator
import functools
import yaml
import math
from collections import defaultdict

In [2]:
%cd ../..

/nfs/homedirs/fuchsgru/MastersThesis


In [3]:
from data.construct import load_data_from_configuration, load_base_data_from_configuration
import configuration
import data.constants as dc

In [17]:
config = configuration.DataConfiguration(
    dataset='ogbn_arxiv', 
    preprocessing='none',
    type='npz',
    #preprocessing='word_embedding',
    #language_model = 'bert-base-uncased',
    #language_model = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    #language_model = 'allenai/longformer-base-4096',
    drop_train_vertices_portion = 0.1,
)

In [18]:
import data.npz

npz_data = np.load(data.npz.NpzDataset.raw_files[config.dataset], allow_pickle=True)
list(npz_data.keys())

['adj_data',
 'adj_indices',
 'adj_indptr',
 'adj_shape',
 'attr_text',
 'features',
 'labels',
 'year',
 'idx_to_class',
 'idx_to_node',
 'mask_train',
 'mask_val',
 'mask_test']

In [19]:
from data.split import _make_base_data
_, _, mask_fixed, mask_non_fixed, x_base, edge_index_base, y_base, vertex_to_idx_base, label_to_idx_base, base_labels, train_labels, \
    left_out_class_labels = _make_base_data(load_base_data_from_configuration(config), config)


In [20]:
edge_index = edge_index_base.copy()
n = int(edge_index.max() + 1)

In [21]:
A = sp.coo_matrix((np.ones(edge_index.shape[1]), edge_index), shape=(n, n), dtype=bool)

In [22]:
from tqdm import tqdm

def k_hop_neighbourhood(A, k, strict=True):
    A = A.tocsc().astype(bool)
    powers = [sp.identity(A.shape[0], dtype=bool, format='csr')]
    for _ in tqdm(range(k), desc='Iterations'):
        powers.append((powers[-1] @ A).astype(bool))
        
    print('To lil...')
    result = powers[-1].tolil()
    
    # for idx, p in enumerate(powers):
        # print(idx, np.where(p.tocsr()[1].todense() > 0)[1])
    
    if strict:
        # Subtract the < k neighbourhoods
        for prev in tqdm(powers[:-1], desc='stricting...'):
            result[prev.nonzero()] = False
            # print(np.where(result.tocsr()[1].todense() > 0)[1])
    print('To csr')
    result = result.tocsr()
    result.eliminate_zeros()
    
    return result



In [23]:
def k_hop_neighbourhood(A, k):
     
    mat = sp.identity(A.shape[0], dtype=bool, format='csr')
    result = sp.identity(A.shape[0], dtype=int, format='csr')
    A = A.tocsc().astype(bool) # csr x csc is fast
    
    # The k-i hop neighbourhood will have a value of 2^i in the resulting matrix
    # Therefore, the biggest power of 2 represents the bfs numbers
    # i.e. entries with a 1 will have a bfs number of k
    for it in tqdm(range(k)):
        mat = (mat @ A).astype(bool)
        result *= 2
        result += mat
        
    result = (result == 1)
    result.eliminate_zeros()
    return result
        
    

In [24]:
three_hops = k_hop_neighbourhood_loop(A, 3)

 33%|███▎      | 1/3 [00:00<00:00,  8.74it/s]

mul
add
mul
add


 67%|██████▋   | 2/3 [00:26<00:15, 15.55s/it]

mul
add


100%|██████████| 3/3 [07:33<00:00, 151.16s/it]


cmp
elim


In [16]:
(k_hop_neighbourhood(A, 4) == k_hop_neighbourhood_loop(A, 4)).todense().all()

Iterations: 100%|██████████| 4/4 [00:00<00:00, 21.42it/s]


To lil...


stricting...: 100%|██████████| 4/4 [00:00<00:00, 12.20it/s]


To csr


100%|██████████| 4/4 [00:00<00:00, 20.78it/s]


Computing result


True

In [12]:
from collections import defaultdict
def k_hop_neighbourhood_loop(A, k):
    
    hops = {i : 0 for i in range(A.shape[0])}
    mat = sp.identity(A.shape[0], dtype=bool, format='csr')
    A = A.tocsc().astype(bool)
    
    for it in range(k):
        mat = (mat @ A).astype(bool)
        for v in tqdm(range(A.shape[0])):
            for k in mat[v, :].nonzero()[0]:
                if k not in hops:
                    hops[k] = it

k_hop_neighbourhood_loop(A, 3)
                
            
    
    

 21%|██▏       | 36318/169343 [02:43<09:58, 222.22it/s]


KeyboardInterrupt: 

In [11]:
K3 = k_hop_neighbourhood(A, 3)

Iterations: 100%|██████████| 3/3 [00:00<00:00, 66.37it/s]


To lil...


stricting...: 100%|██████████| 3/3 [00:00<00:00, 51.61it/s]

To csr


In [246]:

A_2hops = k_hop_neighbourhood(A, 2)

Iterations: 100%|██████████| 2/2 [00:26<00:00, 13.36s/it]


To lil...


stricting...: 100%|██████████| 2/2 [00:09<00:00,  4.72s/it]


To csr


In [230]:
A_2hops_adj_list = {k : set(v) for k, v in get_k_hop_neighbourhood(torch.tensor(edge_index), 2, k_min=2).items()}

In [231]:
A_2hops_adj_list2 = defaultdict(set)
for u, v in zip(*A_2hops.nonzero()):
    A_2hops_adj_list2[u].add(v)


In [232]:
A_2hops_adj_list2[1] - A_2hops_adj_list[1], A_2hops_adj_list[1] - A_2hops_adj_list2[1]

(set(), set())

In [233]:
A_2hops_adj_list2 == A_2hops_adj_list

True

In [226]:
for k in A_2hops_adj_list2.keys():
    if not A_2hops_adj_list2[k] == A_2hops_adj_list[k]:
        print(k)

In [161]:
A_2hops_adj_list[0]

{1200,
 1255,
 1282,
 1288,
 1472,
 1474,
 1477,
 1582,
 1583,
 1668,
 1669,
 1674,
 2242,
 2247,
 2397,
 2677}

In [144]:
A_2hops_adj_list2[0]

{0,
 1200,
 1255,
 1282,
 1288,
 1472,
 1474,
 1477,
 1579,
 1581,
 1582,
 1583,
 1668,
 1669,
 1674,
 2241,
 2242,
 2247,
 2397,
 2677}

In [139]:
np.where(A.tocsr()[0].todense() > 0)[1]

array([1579, 1581, 2241])

In [140]:
A_2hops_adj_list2[0] - A_2hops_adj_list[0]

{0, 1579, 1581, 2241}

In [44]:
test = test.reshape((-1, 1))
A2.multiply(test).nnz, A2.nnz
A2.todense()

matrix([[3., 0., 0., ..., 0., 0., 0.],
        [0., 7., 1., ..., 0., 0., 0.],
        [0., 1., 7., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 2., 0., 0.],
        [0., 0., 0., ..., 0., 3., 0.],
        [0., 0., 0., ..., 0., 0., 1.]])

In [30]:
nnz = np.array(A2.nonzero())

In [31]:
nnz.shape

(2, 212634)

In [9]:
from util import get_k_hop_neighbourhood

In [237]:
A = sp.csr_matrix(np.array([
    [0, 1, 1, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 1],
    [0, 0, 0, 1]
])).astype(bool)

attr = np.array([
    True, False, False, True
])

torch.tensor(A.multiply(attr[None, :]).sum(1)).squeeze()

tensor([0, 1, 2, 1])

In [56]:
(A @ A).todense()

matrix([[ True, False, False,  True],
        [False,  True,  True, False],
        [False,  True,  True,  True],
        [False, False, False,  True]])